In [ ]:
# 필요한 라이브러리 설치
# !pip install PyMuPDF python-docx openai
# pip install python-dotenv

In [ ]:
# =============================================================================
#   라이브러리 임포트
# =============================================================================

import os
import re
import fitz # PDF용
import docx # DOCX용
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [ ]:
# =============================================================================
#   OpenAI API 키 환경변수 설정
# =============================================================================

# .env 파일 로드 (OPENAI_API_KEY를 환경변수로 등록)
load_dotenv()

client = OpenAI()

In [ ]:
# =============================================================================
#   첨부 파일 경로 설정 및 형식 확인
# =============================================================================

# 첨부한 파일 경로
file_path = "../resume/resume.txt"

# 확장자 추출 후 소문자로 변환 (pdf, docx, txt 구분용)
ext = os.path.splitext(file_path)[-1].lower()

In [4]:
# =============================================================================
#   첨부 파일에서 텍스트 추출
# =============================================================================

extracted_text = ""

if ext == ".pdf":
    with fitz.open(file_path) as doc:
        for page in doc:
            extracted_text += page.get_text()
elif ext == ".docx":
    doc = docx.Document(file_path)
    for para in doc.paragraphs:
        extracted_text += para.text + "\n"
elif ext == ".txt":
    with open(file_path, "r", encoding="utf-8") as f:
        extracted_text = f.read()
else:
    raise ValueError(f"지원하지 않는 파일 형식입니다: {ext}")

In [ ]:
# =============================================================================
#   불필요한 정보 제거 및 정제
# =============================================================================

def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # 불필요한 줄바꿈/공백 제거
    text = re.sub(r'Page\s*\d+\s*/\s*\d+', '', text, flags=re.I)  # 페이지 표시 제거
    text = re.sub(r'(\w+@\w+\.\w+)', '[이메일]', text)  # 이메일 마스킹
    text = re.sub(r'(\d{2,4}[-.\s]?\d{2,4}[-.\s]?\d{4})', '[전화번호]', text)  # 전화번호 마스킹
    return text.strip()

extracted_text = preprocess_text(extracted_text)

In [5]:
# =============================================================================
#   추출된 텍스트 일부 미리보기
# =============================================================================

display(Markdown("추출한 텍스트 일부 미리보기"))
display(Markdown(f"```\n{extracted_text[:1000]}...\n```"))

추출한 텍스트 일부 미리보기

```
홍길동
이메일: honggildong@example.com
전화번호: 010-1234-5678

요약
- 5년 이상의 백엔드 개발 경력
- Python, Django, Node.js를 활용한 웹서비스 구축
- AWS 기반 서비스 배포 및 인프라 관리 경험

주요 경력
- ABC주식회사 / 백엔드 개발자 / 2020.01 ~ 현재
  - 대용량 트래픽 처리 시스템 설계 및 구현
  - REST API 서버 개발 및 유지보수

- XYZ스타트업 / 웹 개발자 / 2017.06 ~ 2019.12
  - 사용자 인증 및 결제 모듈 개발
  - 프론트엔드 개발팀과 협업하여 SPA 구축

기술 스택
- 언어: Python, JavaScript, Java
- 프레임워크: Django, Flask, Express
- 데이터베이스: MySQL, PostgreSQL
- 기타: Git, Docker, AWS, Linux

학력
- 대한민국대학교 컴퓨터공학과 학사 졸업 (2017)...
```

In [ ]:
# =============================================================================
#   프롬프트 작성
# =============================================================================

prompt = f"""
너는 채용 전문가이자 평가 컨설턴트야.

아래는 지원자의 이력서 내용이야. 내용을 분석하고 아래 질문에 답변해줘.  
답변은 표를 포함한 Markdown 형식으로 작성하고, 마지막에 지원자의 '직무 적합도 점수(100점 만점)'를 꼭 표시해줘.

## 분석 항목
1) 핵심 경력 및 기술 역량은?
2) 수행한 프로젝트나 업무 중 가장 성과가 좋았던 사례는?
3) 직무에 적합한 소프트스킬(커뮤니케이션, 팀워크 등)은?
4) 보완하거나 개선하면 좋을 점은?
5) 이력서에 기술되지 않은 추가로 궁금한 사항은?

[이력서 내용]
-----------------
{extracted_text}
-----------------
"""

In [ ]:

# =============================================================================
#   OPENAI API 호출
# =============================================================================

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "당신은 전문 인사 담당자입니다."},
        {"role": "user", "content": prompt}
    ]
)

analysis = response.choices[0].message.content

In [8]:

# =============================================================================
#   분석 결과 출력
# =============================================================================

display(Markdown("GPT 분석 결과 (OpenAI)"))
display(Markdown(analysis))

GPT 분석 결과 (OpenAI)

```markdown
1) 지원자의 핵심 경력과 기술 역량은?
   - **핵심 경력**: 5년 이상의 경력을 통해 백엔드 개발에 전문성이 있는 지원자로, 주로 대용량 트래픽 처리 시스템 설계 및 구현과 REST API 서버 개발 및 유지보수에 집중해왔습니다.
   - **기술 역량**: Python, Django, Node.js를 이용한 웹서비스 구축과 AWS를 활용한 서비스 배포 및 인프라 관리 경험을 가지고 있습니다. 데이터베이스 관리(MySQL, PostgreSQL) 및 DevOps 도구(Git, Docker, AWS, Linux)에 능숙합니다.

2) 지원자의 경력 중 가장 강점이 되는 부분은?
   - **대용량 트래픽 처리 시스템 설계 및 구현**: 지원자의 경력 중 ABC주식회사에서 대용량 트래픽 처리 시스템을 설계하고 구현한 경험이 가장 강점인 부분으로 보입니다. 이는 복잡한 문제 해결 능력과 효율적인 시스템 설계 능력을 보여줍니다.
   - **AWS 기반 서비스 운영**: AWS 환경에서 서비스를 배포하고 인프라를 관리한 경험도 중요한 강점입니다. 이 역량은 현재 많은 기업이 클라우드 환경으로 이전하면서 더욱 중요해지고 있습니다.

3) 개선하거나 보완하면 좋을 점은?
   - **프론트엔드 기술 역량 강화**: 과거 XYZ스타트업에서 프론트엔드 개발팀과 협업하여 SPA를 구축한 경험이 있으나, 프론트엔드 기술 스택에 대한 구체적 언급이 없으므로 이 부분에서의 역량이 미흡할 수 있습니다. 보다 원활한 협업과 전체적인 웹 애플리케이션의 이해를 위해 프론트엔드 기술(예: React, Angular, Vue.js)에 대한 지식을 강화하면 좋을 것입니다.
   - **프로젝트 관리 경험**: 현재 이력서에는 팀 리드나 프로젝트 관리 경험이 언급되지 않았습니다. 향후 커리어 발전을 위해 프로젝트 관리를 통한 리더십 경험을 쌓는 것도 추천드립니다.
```
